<a href="https://colab.research.google.com/github/nmalby/Daily-Transaction-Audit-Extractor/blob/main/Daily-Transaction-Audit-Extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tabula-py
!pip install pandas

In [2]:
import io
import re
import tabula
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

from google.colab import files

In [ ]:
uploaded = files.upload()

In [4]:
pdf = io.BytesIO(uploaded[next(iter(uploaded))])
pdf.name = next(iter(uploaded))

In [5]:
sectionCodes = {
    'supervis 1': 205,
    'coordina 1': 206,
    'pockets': 207,
    'backs': 208,
    'bodies': 209,
    'slv/col/fly': 211,
    'fronts': 212,
    'parts': 213,
    'coordina 2': 221,
    'shell prep': 222,
    'therm/mb 1': 223,
    'therm/mb 2': 224,
    'coordina 3': 236,
    'assembly': 237,
    'finishing': 238,
    'hardware': 239,
    'misc lbr': 266,
    'training': 270,
    'engineer': 295,
    'cutting indirect': 305,
    'cutting direct': 310
}

remove_pass_one =   {'Transaction Type':['----------------','Earnings Inquiry','IPR...Labor Code:','Lunch Start','Lunch Stop','Sign-OFF','difications.......', 'Operator Inquiry', 'Bundle Inquiry','Bulk Start','Bundle Quit'],
                     'Problem':['Bun already done','Currently on job','Job not found', 'Not signed on', 'Cur on lunch','Cur on other W/S','Already signd on']}
remove_pass_two =   {#'Code Pay':['18T'],
                     'Trans':['for Emp',"999999"]}
                     #,'Sect':["295","305","310"]}

In [ ]:

top = 100
left = 35
height = 455
width = 715

columnsBounds = {'time': 70, 'trans': 110, 'trans type': 205, 'problem': 300, 'ipr': 320, 'line job': 350, 'labr code': 400, 'split nbr': 430, 'rate fctr': 495, 'sect': 520, 'bundle num': 560, 'qty': 610}

df_raw = tabula.read_pdf(pdf, pages = '002-420', area=[top,left,top+height,left+width], columns=list(columnsBounds.values()), guess=False)
df_final = pd.DataFrame()

date = re.search(r'([0-9]+(\.[0-9]+)+)', pdf.name).group().replace(".", "/")

for item in df_raw:
    
    item = item[~item.isin(remove_pass_one).any(axis='columns')]

    item['Job'] = item['Job'].fillna(method='ffill')

    extracted_id = (item['Transaction Type'].str.extract(r'loyee (\d+)',expand=False)).bfill()
    if extracted_id.tail(1).item():
        item['Nbr'] = item['Nbr'].fillna(extracted_id.tail(1).item())

    df_final = pd.concat([df_final, item], ignore_index=True)

    df_final = df_final[~df_final.isin(remove_pass_two).any(axis='columns')]
    df_final.dropna(subset=['Number'], inplace=True)
df_final.rename(columns={'IPR': 'Date'}, inplace=True)
df_final.rename(columns={'Nbr': 'ID'}, inplace=True)

df_final['Job'].fillna(method='ffill', inplace=True)
df_final['Date'].fillna(value=date, inplace=True)
df_final['ID'].fillna(method='bfill', inplace=True)

df_final.drop(columns=['Sect', 'Sp  Qty', 'SAHAmountEFF'], inplace=True)

# display(df_final)
df_final.to_excel(pdf.name.split('/')[-1].split('.p')[0] + '.xlsx')
